In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Dùng khi chạy trên Colab
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

import torch
from unsloth import FastLanguageModel

2025-09-29 03:29:37.094011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759116577.453030      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759116577.553915      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# # Kiểm tra số dòng trong train.jsonl
# with open('/kaggle/input/processed-data1/train.jsonl', 'r', encoding='utf-8') as f:
#     train_lines = sum(1 for line in f)

# # Kiểm tra số dòng trong valid.jsonl
# with open('/kaggle/input/processed-data1/val.jsonl', 'r', encoding='utf-8') as f:
#     valid_lines = sum(1 for line in f)

# print(f"Số dòng trong train.jsonl: {train_lines}")
# print(f"Số dòng trong valid.jsonl: {valid_lines}")


In [3]:
# Tải model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Thinking-2507-unsloth-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

==((====))==  Unsloth 2025.9.9: Fast Qwen3 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [4]:
# Cấu hình LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [5]:
# Load chat template Qwen3
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen3-thinking",
)

In [6]:
# ==== Load dataset JSONL ====
from datasets import load_dataset

train_dataset = load_dataset("json", data_files="/kaggle/input/processed-data1/train.jsonl", split="train")
val_dataset   = load_dataset("json", data_files="/kaggle/input/processed-data1/val.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
# Map sang hội thoại
def generate_conversation(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    conversations = []
    for ins, inp, out in zip(instructions, inputs, outputs):
        if inp and inp.strip():
            user_prompt = ins.strip() + "\n" + inp.strip()
        else:
            user_prompt = ins.strip()
        conversations.append([
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": out.strip()},
        ])
    return {"conversations": conversations}

train_dataset = train_dataset.map(generate_conversation, batched=True)
val_dataset   = val_dataset.map(generate_conversation, batched=True)

Map:   0%|          | 0/6174 [00:00<?, ? examples/s]

Map:   0%|          | 0/687 [00:00<?, ? examples/s]

In [8]:
# Format thành text input cho model
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(
        convo,
        tokenize=False,
        add_generation_prompt=False
    ) for convo in convos]
    return {"text": texts}

train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
val_dataset   = val_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/6174 [00:00<?, ? examples/s]

Map:   0%|          | 0/687 [00:00<?, ? examples/s]

In [10]:
# ==== Training 500 steps cho GPU T4 15GB (đã sửa lỗi) ====
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import train_on_responses_only

# Config với 500 steps
max_steps = 500

print(f"Training for {max_steps} steps")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 50,
        max_steps = max_steps,
        learning_rate = 5e-5,
        logging_steps = 25,
        eval_steps = 100,
        save_steps = 100,  # Sửa thành bội số của eval_steps
        eval_strategy = "steps",
        save_strategy = "steps",
        load_best_model_at_end = True,
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        report_to = "none",
        max_grad_norm = 0.5,
        fp16 = True,
        bf16 = False,
        dataloader_pin_memory = False,
        gradient_checkpointing = True,
        gradient_checkpointing_kwargs = {"use_reentrant": False},
    ),
)

# Chỉ train trên response
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

# Train
trainer_stats = trainer.train()

Training for 500 steps


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/6174 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/687 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/6174 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/687 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
100,2.162500,2.189562
200,2.099600,2.132513
300,2.064800,2.107159
400,2.033900,2.096127
500,2.058300,2.094072


In [11]:
# Lưu model
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [17]:
import torch
from transformers import TextStreamer
from unsloth import FastLanguageModel

# Load model và tokenizer đã fine-tuned
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/kaggle/working/lora_model",
    max_seq_length = 2048,
    load_in_4bit = True,
)

device = next(model.parameters()).device

def ask(question: str, max_new_tokens: int = 1000):
    # Hội thoại input với chỉ dẫn rõ ràng
    messages = [
        {"role": "user", "content": question + " Trả lời ngắn gọn, chính xác theo y học hiện đại, không bịa đặt."}
    ]

    # Tạo prompt, tắt reasoning (<think>)
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,   # <- tắt <think>
    )

    # Tokenize và đưa lên device
    inputs = tokenizer(text, return_tensors="pt").to(device)

    # Generate với cấu hình an toàn hơn
    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.5,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2,
        no_repeat_ngram_size=3,
        early_stopping=True,
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Chỉ lấy phần sau "assistant"
    if "assistant" in decoded:
        decoded = decoded.split("assistant", 1)[-1].strip()

    return decoded

# Ví dụ test
print("Final output:\n", ask("Hỏi: Rối loạn ăn uống là gì?"))

==((====))==  Unsloth 2025.9.9: Fast Qwen3 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Final output:
 <think>
</think>

Đáp án: Một rối loạn ăn kiêng (EDM) là một nhóm các tình trạng liên quan đến việc tiêu thụ thức ăn và sự thèm ăn bất thường trong đó có nhiều yếu tố tâm lý, xã hội, lạm dụng hóa chất hoặc bệnh tật.Phân loại EDM được phân loại thành 6 nhóm chính: chứng khó chịu về cân nặng, nghiện thực phẩm, rối loạn chế độ ăn kiếng, rối hợp đồng và hành vi tự gây ra.Tình trạng này đã xảy ra ở cả nam giới và nữ giới.Mặc dù nó có thể ảnh hưởng đến mọi người khác nhau từ trẻ em đến già, nhưng nó phổ biến nhất ở tuổi trưởng th

In [18]:
import shutil

# Đường dẫn thư mục bạn muốn nén
folder_path = "/kaggle/working/lora_model"

# Tên file zip sẽ tạo ra
zip_file_path = "/kaggle/working/lora_model.zip"

# Nén thư mục thành file zip
shutil.make_archive(zip_file_path.replace(".zip", ""), 'zip', folder_path)

print("Đã tạo file zip:", zip_file_path)

Đã tạo file zip: /kaggle/working/lora_model.zip
